In [1]:
from pprint import pprint
from IPython.display import display

from fda_ode import *

In [2]:
eps, t, h = symbols(r'\varepsilon, t, h', real=True)
u, u1 = (f(t) for f in symbols('u, u1', cls=Function))

In [3]:
eq = u.diff(t, 2)  + u + eps*u**3
eq

\varepsilon*u(t)**3 + u(t) + Derivative(u(t), (t, 2))

Первый интеграл страница 123

https://disk.yandex.ru/i/ng5ws4p2rYTcQA

In [4]:
eq1 = (u.diff(t)**2/2 + u**2/2 + eps*u**4/4) - h

In [5]:
expand(eq1.diff(t) - eq*u.diff(t))

0

In [6]:
init((u, u1), t, h)
set_clip(8, 7, Rational(0, 1))

In [7]:
expand((T(u, 1)-T(u, -1))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [8]:
def GaussLegendre2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y1/2 + y/2)
    return expand(k1)

In [9]:
def CrankNicolson2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y)
    k2 = f(y1)
    return expand((k1 + k2)/2)

In [10]:
def RungeKutta4(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/2)
    k3 = f(y + h*k2/2)
    k4 = f(y + h*k3)
    return expand((k1 + 2*k2 + 2*k3 + k4)/6)

In [11]:
def DormandPrince5(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    return expand(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84)

def DormandPrince51(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    k7 = f(y + h*(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84))
    return expand(k1*5179/57600 + k3*7571/16695 + k4*393/640 - k5*92097/339200 + k6*187/2100 + k7/40)

In [12]:
def f(y):
    return Matrix([\
        clip_n(y[1]),\
        clip_n(-y[0] - eps*y[0]**3),\
    ])

## Gauss–Legendre method

In [13]:
set_clip(5, 4, Rational(0, 1))
r = GaussLegendre2(f, Matrix([u, u1]))

In [14]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-(u1_{t} - u_{tt})/2

h^2 =>


-(3*u1_{tt} - 2*u_{ttt})/12

h^3 =>


-(2*u1_{ttt} - u_{tttt})/24

- u_{1} + u_{t}
+h\big(
- \frac{u1_{t} - u_{tt}}{2}
\big)
+h^2\big(
- \frac{3 u1_{tt} - 2 u_{ttt}}{12}
\big)
+h^3\big(
- \frac{2 u1_{ttt} - u_{tttt}}{24}
\big)


In [15]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u_{t}/2 + (u1_{tt} + u_{t})/2

h^2 =>


3*\varepsilon*u*(u*u_{tt} + u_{t}**2)/4 + (2*u1_{ttt} + 3*u_{tt})/12

h^3 =>


\varepsilon*(2*u**2*u_{ttt} + 6*u*u_{tt}*u_{t} + u_{t}**3)/8 + (u1_{tttt} + 2*u_{ttt})/24

\varepsilon u^{3} + u + u1_{t}
+h\big(
\frac{3 \varepsilon u^{2} u_{t}}{2} + \frac{u1_{tt} + u_{t}}{2}
\big)
+h^2\big(
\frac{3 \varepsilon u \left(u u_{tt} + u_{t}^{2}\right)}{4} + \frac{2 u1_{ttt} + 3 u_{tt}}{12}
\big)
+h^3\big(
\frac{\varepsilon \left(2 u^{2} u_{ttt} + 6 u u_{tt} u_{t} + u_{t}^{3}\right)}{8} + \frac{u1_{tttt} + 2 u_{ttt}}{24}
\big)


In [16]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


\varepsilon*u**2*u1/4 + u1/12

h^3 =>


0

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
\frac{\varepsilon u^{2} u_{1}}{4} + \frac{u_{1}}{12}
\big)
+h^3\big(
0
\big)


In [17]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


-\varepsilon**2*u**5/4 - \varepsilon*u*(4*u**2 + 3*u1**2)/12 - u/12

h^3 =>


0

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
- \frac{\varepsilon^{2} u^{5}}{4} - \frac{\varepsilon u \left(4 u^{2} + 3 u_{1}^{2}\right)}{12} - \frac{u}{12}
\big)
+h^3\big(
0
\big)


In [18]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3, eps)
prnlatex(F3, eps)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


\varepsilon*u**2*(u*u_{tt} + 3*u_{t}**2)/2 + (u*u_{tt} + u1*u1_{tt} + u1_{t}**2 + u_{t}**2)/2

h^3 =>


\varepsilon*u*(u**2*u_{ttt} + 9*u*u_{tt}*u_{t} + 6*u_{t}**3)/6 + (u*u_{ttt} + u1*u1_{ttt} + 3*u1_{tt}*u1_{t} + 3*u_{tt}*u_{t})/6

0
+h\big(
\varepsilon u^{3} u_{t} + u u_{t} + u_{1} u1_{t}
\big)
+h^2\big(
\frac{\varepsilon u^{2} \left(u u_{tt} + 3 u_{t}^{2}\right)}{2} + \frac{u u_{tt} + u_{1} u1_{tt} + u1_{t}^{2} + u_{t}^{2}}{2}
\big)
+h^3\big(
\frac{\varepsilon u \left(u^{2} u_{ttt} + 9 u u_{tt} u_{t} + 6 u_{t}^{3}\right)}{6} + \frac{u u_{ttt} + u_{1} u1_{ttt} + 3 u1_{tt} u1_{t} + 3 u_{tt} u_{t}}{6}
\big)


In [19]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [f1, f2], head=False)
prn(f3, eps)
prnlatex(f3, eps)

h^0 =>


0

h^1 =>


0

h^2 =>


0

h^3 =>


\varepsilon*u*u1**3/4

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
\frac{\varepsilon u u_{1}^{3}}{4}
\big)


## Crank–Nicolson method

In [20]:
set_clip(5, 4, Rational(0, 1))
r = CrankNicolson2(f, Matrix([u, u1]))

In [21]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-(u1_{t} - u_{tt})/2

h^2 =>


-(3*u1_{tt} - 2*u_{ttt})/12

h^3 =>


-(2*u1_{ttt} - u_{tttt})/24

- u_{1} + u_{t}
+h\big(
- \frac{u1_{t} - u_{tt}}{2}
\big)
+h^2\big(
- \frac{3 u1_{tt} - 2 u_{ttt}}{12}
\big)
+h^3\big(
- \frac{2 u1_{ttt} - u_{tttt}}{24}
\big)


In [22]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u_{t}/2 + (u1_{tt} + u_{t})/2

h^2 =>


3*\varepsilon*u*(u*u_{tt} + 2*u_{t}**2)/4 + (2*u1_{ttt} + 3*u_{tt})/12

h^3 =>


\varepsilon*(u**2*u_{ttt} + 6*u*u_{tt}*u_{t} + 2*u_{t}**3)/4 + (u1_{tttt} + 2*u_{ttt})/24

\varepsilon u^{3} + u + u1_{t}
+h\big(
\frac{3 \varepsilon u^{2} u_{t}}{2} + \frac{u1_{tt} + u_{t}}{2}
\big)
+h^2\big(
\frac{3 \varepsilon u \left(u u_{tt} + 2 u_{t}^{2}\right)}{4} + \frac{2 u1_{ttt} + 3 u_{tt}}{12}
\big)
+h^3\big(
\frac{\varepsilon \left(u^{2} u_{ttt} + 6 u u_{tt} u_{t} + 2 u_{t}^{3}\right)}{4} + \frac{u1_{tttt} + 2 u_{ttt}}{24}
\big)


In [23]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


\varepsilon*u**2*u1/4 + u1/12

h^3 =>


0

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
\frac{\varepsilon u^{2} u_{1}}{4} + \frac{u_{1}}{12}
\big)
+h^3\big(
0
\big)


In [24]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


-\varepsilon**2*u**5/4 - \varepsilon*u*(2*u**2 - 3*u1**2)/6 - u/12

h^3 =>


0

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
- \frac{\varepsilon^{2} u^{5}}{4} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{6} - \frac{u}{12}
\big)
+h^3\big(
0
\big)


In [25]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3, eps)
prnlatex(F3, eps)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


\varepsilon*u**2*(u*u_{tt} + 3*u_{t}**2)/2 + (u*u_{tt} + u1*u1_{tt} + u1_{t}**2 + u_{t}**2)/2

h^3 =>


\varepsilon*u*(u**2*u_{ttt} + 9*u*u_{tt}*u_{t} + 6*u_{t}**3)/6 + (u*u_{ttt} + u1*u1_{ttt} + 3*u1_{tt}*u1_{t} + 3*u_{tt}*u_{t})/6

0
+h\big(
\varepsilon u^{3} u_{t} + u u_{t} + u_{1} u1_{t}
\big)
+h^2\big(
\frac{\varepsilon u^{2} \left(u u_{tt} + 3 u_{t}^{2}\right)}{2} + \frac{u u_{tt} + u_{1} u1_{tt} + u1_{t}^{2} + u_{t}^{2}}{2}
\big)
+h^3\big(
\frac{\varepsilon u \left(u^{2} u_{ttt} + 9 u u_{tt} u_{t} + 6 u_{t}^{3}\right)}{6} + \frac{u u_{ttt} + u_{1} u1_{ttt} + 3 u1_{tt} u1_{t} + 3 u_{tt} u_{t}}{6}
\big)


In [26]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [f1, f2], head=False)
prn(f3, eps)
prnlatex(f3, eps)

h^0 =>


0

h^1 =>


0

h^2 =>


0

h^3 =>


-\varepsilon*u*u1**3/2

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
- \frac{\varepsilon u u_{1}^{3}}{2}
\big)


## "original" Runge–Kutta method

In [27]:
set_clip(7, 6, Rational(0, 1))
r = RungeKutta4(f, Matrix([u, u1]))

In [28]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*u**3/2 + (u + u_{tt})/2

h^2 =>


...

h^4 =>


-\varepsilon**2*u**4*u1/8 - \varepsilon*u1*(3*u**2 - u1**2)/24 + u_{ttttt}/120

h^5 =>


\varepsilon**3*u**7/32 + \varepsilon**2*u**3*(2*u**2 - u1**2)/32 + \varepsilon*u*(u - u1)*(u + u1)/32 + u_{tttttt}/720

- u_{1} + u_{t}
+h\big(
\frac{\varepsilon u^{3}}{2} + \frac{u + u_{tt}}{2}
\big)
+h^2\big(
\frac{\varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u_{ttt}}{6}
\big)
+h^3\big(
- \frac{\varepsilon^{2} u^{5}}{8} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{12} - \frac{u - u_{tttt}}{24}
\big)
+h^4\big(
- \frac{\varepsilon^{2} u^{4} u_{1}}{8} - \frac{\varepsilon u_{1} \cdot \left(3 u^{2} - u_{1}^{2}\right)}{24} + \frac{u_{ttttt}}{120}
\big)
+h^5\big(
\frac{\varepsilon^{3} u^{7}}{32} + \frac{\varepsilon^{2} u^{3} \cdot \left(2 u^{2} - u_{1}^{2}\right)}{32} + \frac{\varepsilon u \left(u - u_{1}\right) \left(u + u_{1}\right)}{32} + \frac{u_{tttttt}}{720}
\big)


In [29]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u1/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^4 =>


3*\varepsilon**3*u**7/16 + \varepsilon**2*u**3*(3*u**2 - 10*u1**2)/8 + \varepsilon*u*(3*u**2 - 10*u1**2)/16 + u1_{ttttt}/120

h^5 =>


17*\varepsilon**3*u**6*u1/32 + \varepsilon**2*u**2*u1*(26*u**2 - 25*u1**2)/32 + \varepsilon*u1*(27*u**2 - 13*u1**2)/96 + u1_{tttttt}/720

\varepsilon u^{3} + u + u1_{t}
+h\big(
\frac{3 \varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u1_{tt}}{2}
\big)
+h^2\big(
- \frac{\varepsilon^{2} u^{5}}{2} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{3} - \frac{u - u1_{ttt}}{6}
\big)
+h^3\big(
- \frac{9 \varepsilon^{2} u^{4} u_{1}}{8} - \frac{\varepsilon u_{1} \cdot \left(2 u - u_{1}\right) \left(2 u + u_{1}\right)}{4} - \frac{u_{1} - u1_{tttt}}{24}
\big)
+h^4\big(
\frac{3 \varepsilon^{3} u^{7}}{16} + \frac{\varepsilon^{2} u^{3} \cdot \left(3 u^{2} - 10 u_{1}^{2}\right)}{8} + \frac{\varepsilon u \left(3 u^{2} - 10 u_{1}^{2}\right)}{16} + \frac{u1_{ttttt}}{120}
\big)
+h^5\big(
\frac{17 \varepsilon^{3} u^{6} u_{1}}{32} + \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(26 u^{2} - 25 u_{1}^{2}\right)}{32} + \frac{\varepsilon u_{1} \cdot \left(27 u^{2} - 13 u_{1}^{2}\right)}{96} + \frac{u1_{tttttt}}{720}
\big)


In [30]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


\varepsilon**2*u**4*u1/10 + \varepsilon*u1*(3*u - u1)*(3*u + u1)/120 + u1/120

h^5 =>


\varepsilon**3*u**7/16 + \varepsilon**2*u**3*(10*u**2 + 3*u1**2)/96 + \varepsilon*u*(14*u**2 + 3*u1**2)/288 + u/144

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{\varepsilon^{2} u^{4} u_{1}}{10} + \frac{\varepsilon u_{1} \cdot \left(3 u - u_{1}\right) \left(3 u + u_{1}\right)}{120} + \frac{u_{1}}{120}
\big)
+h^5\big(
\frac{\varepsilon^{3} u^{7}}{16} + \frac{\varepsilon^{2} u^{3} \cdot \left(10 u^{2} + 3 u_{1}^{2}\right)}{96} + \frac{\varepsilon u \left(14 u^{2} + 3 u_{1}^{2}\right)}{288} + \frac{u}{144}
\big)


In [31]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-3*\varepsilon**3*u**7/80 - \varepsilon**2*u**3*(u**2 + 4*u1**2)/20 - \varepsilon*u*(5*u**2 + 18*u1**2)/240 - u/120

h^5 =>


-\varepsilon**2*u**2*u1*(2*u**2 - u1**2)/32 - \varepsilon*u1**3/96 + u1/144

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{3 \varepsilon^{3} u^{7}}{80} - \frac{\varepsilon^{2} u^{3} \left(u^{2} + 4 u_{1}^{2}\right)}{20} - \frac{\varepsilon u \left(5 u^{2} + 18 u_{1}^{2}\right)}{240} - \frac{u}{120}
\big)
+h^5\big(
- \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(2 u^{2} - u_{1}^{2}\right)}{32} - \frac{\varepsilon u_{1}^{3}}{96} + \frac{u_{1}}{144}
\big)


In [32]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3, eps)
prnlatex(F3, eps)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


...

h^4 =>


\varepsilon*(u**3*u_{tttt} + 12*u**2*u_{ttt}*u_{t} + 9*u**2*u_{tt}**2 + 36*u*u_{tt}*u_{t}**2 + 6*u_{t}**4)/24 + (u*u_{tttt} + u1*u1_{tttt} + 4*u1_{ttt}*u1_{t} + 3*u1_{tt}**2 + 4*u_{ttt}*u_{t} + 3*u_{tt}**2)/24

h^5 =>


\varepsilon*(u**3*u_{ttttt} + 15*u**2*u_{tttt}*u_{t} + 30*u**2*u_{ttt}*u_{tt} + 60*u*u_{ttt}*u_{t}**2 + 90*u*u_{tt}**2*u_{t} + 60*u_{tt}*u_{t}**3)/120 + (u*u_{ttttt} + u1*u1_{ttttt} + 5*u1_{tttt}*u1_{t} + 10*u1_{ttt}*u1_{tt} + 5*u_{tttt}*u_{t} + 10*u_{ttt}*u_{tt})/120

0
+h\big(
\varepsilon u^{3} u_{t} + u u_{t} + u_{1} u1_{t}
\big)
+h^2\big(
\frac{\varepsilon u^{2} \left(u u_{tt} + 3 u_{t}^{2}\right)}{2} + \frac{u u_{tt} + u_{1} u1_{tt} + u1_{t}^{2} + u_{t}^{2}}{2}
\big)
+h^3\big(
\frac{\varepsilon u \left(u^{2} u_{ttt} + 9 u u_{tt} u_{t} + 6 u_{t}^{3}\right)}{6} + \frac{u u_{ttt} + u_{1} u1_{ttt} + 3 u1_{tt} u1_{t} + 3 u_{tt} u_{t}}{6}
\big)
+h^4\big(
\frac{\varepsilon \left(u^{3} u_{tttt} + 12 u^{2} u_{ttt} u_{t} + 9 u^{2} u_{tt}^{2} + 36 u u_{tt} u_{t}^{2} + 6 u_{t}^{4}\right)}{24} + \frac{u u_{tttt} + u_{1} u1_{tttt} + 4 u1_{ttt} u1_{t} + 3 u1_{tt}^{2} + 4 u_{ttt} u_{t} + 3 u_{tt}^{2}}{24}
\big)
+h^5\big(
\frac{\varepsilon \left(u^{3} u_{ttttt} + 15 u^{2} u_{tttt} u_{t} + 30 u^{2} u_{ttt} u_{tt} + 60 u u_{ttt} u_{t}^{2} + 90 u u_{tt}^{2} u_{t} + 60 u_{tt} u_{t}^{3}\right)}{120} + \frac{u u_{ttttt} + u_{1} u1_{ttttt} + 5 u1_{tttt} u1_{t} + 10 u1_{ttt} u1_{tt} + 5 u_{tttt} u_{t} + 10 u_{ttt} u_{tt}}{120}
\big)


In [33]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [f1, f2], head=False)
prn(f3, eps)
prnlatex(f3, eps)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


-\varepsilon**3*u**7*u1/16 - \varepsilon**2*u**3*u1*(3*u**2 - 5*u1**2)/24 - \varepsilon*u*u1*(3*u**2 - 4*u1**2)/48

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{\varepsilon^{3} u^{7} u_{1}}{16} - \frac{\varepsilon^{2} u^{3} u_{1} \cdot \left(3 u^{2} - 5 u_{1}^{2}\right)}{24} - \frac{\varepsilon u u_{1} \cdot \left(3 u^{2} - 4 u_{1}^{2}\right)}{48}
\big)


## Dormand–Prince method 5 order

In [34]:
set_clip(8, 7, Rational(0, 1))
r = DormandPrince5(f, Matrix([u, u1]))

In [35]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*u**3/2 + (u + u_{tt})/2

h^2 =>


...

h^5 =>


289*\varepsilon**3*u**7/7200 + \varepsilon**2*u**3*(271*u**2 - 631*u1**2)/3600 + \varepsilon*u*(265*u**2 - 662*u1**2)/7200 + (6*u + 5*u_{tttttt})/3600

h^6 =>


54719*\varepsilon**3*u**6*u1/648000 + \varepsilon**2*u**2*u1*(40295*u**2 - 23184*u1**2)/324000 + \varepsilon*u1*(25871*u**2 - 7296*u1**2)/648000 + u_{ttttttt}/5040

- u_{1} + u_{t}
+h\big(
\frac{\varepsilon u^{3}}{2} + \frac{u + u_{tt}}{2}
\big)
+h^2\big(
\frac{\varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u_{ttt}}{6}
\big)
+h^3\big(
- \frac{\varepsilon^{2} u^{5}}{8} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{12} - \frac{u - u_{tttt}}{24}
\big)
+h^4\big(
- \frac{9 \varepsilon^{2} u^{4} u_{1}}{40} - \frac{\varepsilon u_{1} \cdot \left(2 u - u_{1}\right) \left(2 u + u_{1}\right)}{20} - \frac{u_{1} - u_{ttttt}}{120}
\big)
+h^5\big(
\frac{289 \varepsilon^{3} u^{7}}{7200} + \frac{\varepsilon^{2} u^{3} \cdot \left(271 u^{2} - 631 u_{1}^{2}\right)}{3600} + \frac{\varepsilon u \left(265 u^{2} - 662 u_{1}^{2}\right)}{7200} + \frac{6 u + 5 u_{tttttt}}{3600}
\big)
+h^6\big(
\frac{54719 \varepsilon^{3} u^{6} u_{1}}{648000} + \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(40295 u^{2} - 23184 u_{1}^{2}\right)}{324000} + \frac{\varepsilon u_{1} \cdot \left(25871 u^{2} - 7296 u_{1}^{2}\right)}{648000} + \frac{u_{ttttttt}}{5040}
\big)


In [36]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u1/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^5 =>


4427*\varepsilon**3*u**6*u1/7200 + \varepsilon**2*u**2*u1*(3197*u**2 - 1890*u1**2)/3600 + \varepsilon*u1*(415*u**2 - 132*u1**2)/1440 + (6*u1 + 5*u1_{tttttt})/3600

h^6 =>


-296101*\varepsilon**4*u**9/3888000 - \varepsilon**3*u**5*(2703107*u**2 - 8601660*u1**2)/14256000 - \varepsilon**2*u*(2149103*u**4 - 9610752*u**2*u1**2 + 2149488*u1**4)/14256000 - \varepsilon*u*(1595099*u**2 - 4480524*u1**2)/42768000 + u1_{ttttttt}/5040

\varepsilon u^{3} + u + u1_{t}
+h\big(
\frac{3 \varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u1_{tt}}{2}
\big)
+h^2\big(
- \frac{\varepsilon^{2} u^{5}}{2} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{3} - \frac{u - u1_{ttt}}{6}
\big)
+h^3\big(
- \frac{9 \varepsilon^{2} u^{4} u_{1}}{8} - \frac{\varepsilon u_{1} \cdot \left(2 u - u_{1}\right) \left(2 u + u_{1}\right)}{4} - \frac{u_{1} - u1_{tttt}}{24}
\big)
+h^4\big(
\frac{9 \varepsilon^{3} u^{7}}{40} + \frac{\varepsilon^{2} u^{3} \cdot \left(17 u^{2} - 42 u_{1}^{2}\right)}{40} + \frac{\varepsilon u \left(25 u^{2} - 66 u_{1}^{2}\right)}{120} + \frac{u + u1_{ttttt}}{120}
\big)
+h^5\big(
\frac{4427 \varepsilon^{3} u^{6} u_{1}}{7200} + \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(3197 u^{2} - 1890 u_{1}^{2}\right)}{3600} + \frac{\varepsilon u_{1} \cdot \left(415 u^{2} - 132 u_{1}^{2}\right)}{1440} + \frac{6 u_{1} + 5 u1_{tttttt}}{3600}
\big)
+h^6\big(
- \frac{296101 \varepsilon^{4} u^{9}}{3888000} - \frac{\varepsilon^{3} u^{5} \

In [37]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


19*\varepsilon**3*u**7/7200 + \varepsilon**2*u**3*(4*u - u1)*(4*u + u1)/3600 + \varepsilon*u*(15*u**2 - 2*u1**2)/7200 + u/3600

h^6 =>


-8911*\varepsilon**3*u**6*u1/648000 - \varepsilon**2*u**2*u1*(32575*u**2 - 8757*u1**2)/2268000 - \varepsilon*u1*(22213*u**2 - 8958*u1**2)/4536000 - u1/2100

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{19 \varepsilon^{3} u^{7}}{7200} + \frac{\varepsilon^{2} u^{3} \cdot \left(4 u - u_{1}\right) \left(4 u + u_{1}\right)}{3600} + \frac{\varepsilon u \left(15 u^{2} - 2 u_{1}^{2}\right)}{7200} + \frac{u}{3600}
\big)
+h^6\big(
- \frac{8911 \varepsilon^{3} u^{6} u_{1}}{648000} - \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(32575 u^{2} - 8757 u_{1}^{2}\right)}{2268000} - \frac{\varepsilon u_{1} \cdot \left(22213 u^{2} - 8958 u_{1}^{2}\right)}{4536000} - \frac{u_{1}}{2100}
\big)


In [38]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


17*\varepsilon**3*u**6*u1/7200 + \varepsilon**2*u**4*u1/1800 + \varepsilon*u**2*u1/1440 + u1/3600

h^6 =>


64079*\varepsilon**4*u**9/3888000 + \varepsilon**3*u**5*(3404731*u**2 - 15380820*u1**2)/99792000 + \varepsilon**2*u*(2303059*u**4 - 9873456*u**2*u1**2 - 77616*u1**4)/99792000 + \varepsilon*u*(1771627*u**2 - 5238612*u1**2)/299376000 + u/2100

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{17 \varepsilon^{3} u^{6} u_{1}}{7200} + \frac{\varepsilon^{2} u^{4} u_{1}}{1800} + \frac{\varepsilon u^{2} u_{1}}{1440} + \frac{u_{1}}{3600}
\big)
+h^6\big(
\frac{64079 \varepsilon^{4} u^{9}}{3888000} + \frac{\varepsilon^{3} u^{5} \cdot \left(3404731 u^{2} - 15380820 u_{1}^{2}\right)}{99792000} + \frac{\varepsilon^{2} u \left(2303059 u^{4} - 9873456 u^{2} u_{1}^{2} - 77616 u_{1}^{4}\right)}{99792000} + \frac{\varepsilon u \left(1771627 u^{2} - 5238612 u_{1}^{2}\right)}{299376000} + \frac{u}{2100}
\big)


In [39]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3, eps)
prnlatex(F3, eps)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


...

h^5 =>


\varepsilon*(u**3*u_{ttttt} + 15*u**2*u_{tttt}*u_{t} + 30*u**2*u_{ttt}*u_{tt} + 60*u*u_{ttt}*u_{t}**2 + 90*u*u_{tt}**2*u_{t} + 60*u_{tt}*u_{t}**3)/120 + (u*u_{ttttt} + u1*u1_{ttttt} + 5*u1_{tttt}*u1_{t} + 10*u1_{ttt}*u1_{tt} + 5*u_{tttt}*u_{t} + 10*u_{ttt}*u_{tt})/120

h^6 =>


\varepsilon*(u**3*u_{tttttt} + 18*u**2*u_{ttttt}*u_{t} + 45*u**2*u_{tttt}*u_{tt} + 30*u**2*u_{ttt}**2 + 90*u*u_{tttt}*u_{t}**2 + 360*u*u_{ttt}*u_{tt}*u_{t} + 90*u*u_{tt}**3 + 120*u_{ttt}*u_{t}**3 + 270*u_{tt}**2*u_{t}**2)/720 + (u*u_{tttttt} + u1*u1_{tttttt} + 6*u1_{ttttt}*u1_{t} + 15*u1_{tttt}*u1_{tt} + 10*u1_{ttt}**2 + 6*u_{ttttt}*u_{t} + 15*u_{tttt}*u_{tt} + 10*u_{ttt}**2)/720

0
+h\big(
\varepsilon u^{3} u_{t} + u u_{t} + u_{1} u1_{t}
\big)
+h^2\big(
\frac{\varepsilon u^{2} \left(u u_{tt} + 3 u_{t}^{2}\right)}{2} + \frac{u u_{tt} + u_{1} u1_{tt} + u1_{t}^{2} + u_{t}^{2}}{2}
\big)
+h^3\big(
\frac{\varepsilon u \left(u^{2} u_{ttt} + 9 u u_{tt} u_{t} + 6 u_{t}^{3}\right)}{6} + \frac{u u_{ttt} + u_{1} u1_{ttt} + 3 u1_{tt} u1_{t} + 3 u_{tt} u_{t}}{6}
\big)
+h^4\big(
\frac{\varepsilon \left(u^{3} u_{tttt} + 12 u^{2} u_{ttt} u_{t} + 9 u^{2} u_{tt}^{2} + 36 u u_{tt} u_{t}^{2} + 6 u_{t}^{4}\right)}{24} + \frac{u u_{tttt} + u_{1} u1_{tttt} + 4 u1_{ttt} u1_{t} + 3 u1_{tt}^{2} + 4 u_{ttt} u_{t} + 3 u_{tt}^{2}}{24}
\big)
+h^5\big(
\frac{\varepsilon \left(u^{3} u_{ttttt} + 15 u^{2} u_{tttt} u_{t} + 30 u^{2} u_{ttt} u_{tt} + 60 u u_{ttt} u_{t}^{2} + 90 u u_{tt}^{2} u_{t} + 60 u_{tt} u_{t}^{3}\right)}{120} + \frac{u u_{ttttt} + u_{1} u1_{ttttt} + 5 u1_{tttt} u1_{t} + 10 u1_{ttt} u1_{tt} + 5 u_{tttt} u_{t} + 10 u_{ttt} u_{tt}}{120}
\big)
+h^6\big(
\frac{\varepsilon \left(u^{

In [40]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [f1, f2], head=False)
prn(f3, eps)
prnlatex(f3, eps)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^5 =>


0

h^6 =>


-19*\varepsilon**4*u**10/7200 - \varepsilon**3*u**6*(17*u**2 + 5*u1**2)/2400 - 47*\varepsilon**2*u**6/7200 - \varepsilon*u**2*(17*u**2 + 3*u1**2)/7200 - (u**2 + u1**2)/3600

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
0
\big)
+h^6\big(
- \frac{19 \varepsilon^{4} u^{10}}{7200} - \frac{\varepsilon^{3} u^{6} \cdot \left(17 u^{2} + 5 u_{1}^{2}\right)}{2400} - \frac{47 \varepsilon^{2} u^{6}}{7200} - \frac{\varepsilon u^{2} \cdot \left(17 u^{2} + 3 u_{1}^{2}\right)}{7200} - \frac{u^{2} + u_{1}^{2}}{3600}
\big)


## Dormand–Prince method 4 order

In [41]:
set_clip(7, 6, Rational(0, 1))
r = DormandPrince51(f, Matrix([u, u1]))

In [42]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


\varepsilon*u**3/2 + (u + u_{tt})/2

h^2 =>


...

h^4 =>


-1873*\varepsilon**2*u**4*u1/8000 - \varepsilon*u1*(12402*u**2 - 3037*u1**2)/60000 - (1097*u1 - 1000*u_{ttttt})/120000

h^5 =>


137419*\varepsilon**3*u**7/3600000 + \varepsilon**2*u**3*(130174*u**2 - 351655*u1**2)/1800000 + \varepsilon*u*(127759*u**2 - 359702*u1**2)/3600000 + (483*u + 500*u_{tttttt})/360000

- u_{1} + u_{t}
+h\big(
\frac{\varepsilon u^{3}}{2} + \frac{u + u_{tt}}{2}
\big)
+h^2\big(
\frac{\varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u_{ttt}}{6}
\big)
+h^3\big(
- \frac{\varepsilon^{2} u^{5}}{8} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{12} - \frac{u - u_{tttt}}{24}
\big)
+h^4\big(
- \frac{1873 \varepsilon^{2} u^{4} u_{1}}{8000} - \frac{\varepsilon u_{1} \cdot \left(12402 u^{2} - 3037 u_{1}^{2}\right)}{60000} - \frac{1097 u_{1} - 1000 u_{ttttt}}{120000}
\big)
+h^5\big(
\frac{137419 \varepsilon^{3} u^{7}}{3600000} + \frac{\varepsilon^{2} u^{3} \cdot \left(130174 u^{2} - 351655 u_{1}^{2}\right)}{1800000} + \frac{\varepsilon u \left(127759 u^{2} - 359702 u_{1}^{2}\right)}{3600000} + \frac{483 u + 500 u_{tttttt}}{360000}
\big)


In [43]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


3*\varepsilon*u**2*u1/2 + (u1 + u1_{tt})/2

h^2 =>


...

h^4 =>


20887*\varepsilon**3*u**7/90000 + \varepsilon**2*u**3*(157223*u**2 - 373412*u1**2)/360000 + \varepsilon*u*(38483*u**2 - 98188*u1**2)/180000 + (1097*u + 1000*u1_{ttttt})/120000

h^5 =>


3958063*\varepsilon**3*u**6*u1/6480000 + \varepsilon**2*u**2*u1*(7170031*u**2 - 4162923*u1**2)/8100000 + \varepsilon*u1*(9281039*u**2 - 2807196*u1**2)/32400000 + (483*u1 + 500*u1_{tttttt})/360000

\varepsilon u^{3} + u + u1_{t}
+h\big(
\frac{3 \varepsilon u^{2} u_{1}}{2} + \frac{u_{1} + u1_{tt}}{2}
\big)
+h^2\big(
- \frac{\varepsilon^{2} u^{5}}{2} - \frac{\varepsilon u \left(2 u^{2} - 3 u_{1}^{2}\right)}{3} - \frac{u - u1_{ttt}}{6}
\big)
+h^3\big(
- \frac{9 \varepsilon^{2} u^{4} u_{1}}{8} - \frac{\varepsilon u_{1} \cdot \left(2 u - u_{1}\right) \left(2 u + u_{1}\right)}{4} - \frac{u_{1} - u1_{tttt}}{24}
\big)
+h^4\big(
\frac{20887 \varepsilon^{3} u^{7}}{90000} + \frac{\varepsilon^{2} u^{3} \cdot \left(157223 u^{2} - 373412 u_{1}^{2}\right)}{360000} + \frac{\varepsilon u \left(38483 u^{2} - 98188 u_{1}^{2}\right)}{180000} + \frac{1097 u + 1000 u1_{ttttt}}{120000}
\big)
+h^5\big(
\frac{3958063 \varepsilon^{3} u^{6} u_{1}}{6480000} + \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(7170031 u^{2} - 4162923 u_{1}^{2}\right)}{8100000} + \frac{\varepsilon u_{1} \cdot \left(9281039 u^{2} - 2807196 u_{1}^{2}\right)}{32400000} + \frac{483 u_{1} + 500 u1_{tttttt}}{360000}
\big)


In [44]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-73*\varepsilon**2*u**4*u1/8000 - \varepsilon*u1*(402*u**2 - 37*u1**2)/60000 - 97*u1/120000

h^5 =>


-13373*\varepsilon**3*u**7/1800000 - \varepsilon**2*u**3*(11063*u**2 + 6805*u1**2)/900000 - \varepsilon*u*(10293*u**2 + 5186*u1**2)/1800000 - 77*u/90000

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{73 \varepsilon^{2} u^{4} u_{1}}{8000} - \frac{\varepsilon u_{1} \cdot \left(402 u^{2} - 37 u_{1}^{2}\right)}{60000} - \frac{97 u_{1}}{120000}
\big)
+h^5\big(
- \frac{13373 \varepsilon^{3} u^{7}}{1800000} - \frac{\varepsilon^{2} u^{3} \cdot \left(11063 u^{2} + 6805 u_{1}^{2}\right)}{900000} - \frac{\varepsilon u \left(10293 u^{2} + 5186 u_{1}^{2}\right)}{1800000} - \frac{77 u}{90000}
\big)


In [45]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*u**3 + u + u1_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


637*\varepsilon**3*u**7/90000 + \varepsilon**2*u**3*(4223*u**2 + 4588*u1**2)/360000 + \varepsilon*u*(983*u**2 + 812*u1**2)/180000 + 97*u/120000

h^5 =>


-44393*\varepsilon**3*u**6*u1/1620000 - \varepsilon**2*u**2*u1*(469708*u**2 + 115551*u1**2)/16200000 - \varepsilon*u1*(150518*u**2 - 49857*u1**2)/16200000 - 77*u1/90000

\varepsilon u^{3} + u + u1_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{637 \varepsilon^{3} u^{7}}{90000} + \frac{\varepsilon^{2} u^{3} \cdot \left(4223 u^{2} + 4588 u_{1}^{2}\right)}{360000} + \frac{\varepsilon u \left(983 u^{2} + 812 u_{1}^{2}\right)}{180000} + \frac{97 u}{120000}
\big)
+h^5\big(
- \frac{44393 \varepsilon^{3} u^{6} u_{1}}{1620000} - \frac{\varepsilon^{2} u^{2} u_{1} \cdot \left(469708 u^{2} + 115551 u_{1}^{2}\right)}{16200000} - \frac{\varepsilon u_{1} \cdot \left(150518 u^{2} - 49857 u_{1}^{2}\right)}{16200000} - \frac{77 u_{1}}{90000}
\big)


In [46]:
F3 = clip((T(u1, 1)**2/2 + T(u, 1)**2/2 + eps*T(u, 1)**4/4) -\
          (T(u1, 0)**2/2 + T(u, 0)**2/2 + eps*T(u, 0)**4/4))
prn(F3, eps)
prnlatex(F3, eps)

h^0 =>


0

h^1 =>


\varepsilon*u**3*u_{t} + u*u_{t} + u1*u1_{t}

h^2 =>


...

h^4 =>


\varepsilon*(u**3*u_{tttt} + 12*u**2*u_{ttt}*u_{t} + 9*u**2*u_{tt}**2 + 36*u*u_{tt}*u_{t}**2 + 6*u_{t}**4)/24 + (u*u_{tttt} + u1*u1_{tttt} + 4*u1_{ttt}*u1_{t} + 3*u1_{tt}**2 + 4*u_{ttt}*u_{t} + 3*u_{tt}**2)/24

h^5 =>


\varepsilon*(u**3*u_{ttttt} + 15*u**2*u_{tttt}*u_{t} + 30*u**2*u_{ttt}*u_{tt} + 60*u*u_{ttt}*u_{t}**2 + 90*u*u_{tt}**2*u_{t} + 60*u_{tt}*u_{t}**3)/120 + (u*u_{ttttt} + u1*u1_{ttttt} + 5*u1_{tttt}*u1_{t} + 10*u1_{ttt}*u1_{tt} + 5*u_{tttt}*u_{t} + 10*u_{ttt}*u_{tt})/120

0
+h\big(
\varepsilon u^{3} u_{t} + u u_{t} + u_{1} u1_{t}
\big)
+h^2\big(
\frac{\varepsilon u^{2} \left(u u_{tt} + 3 u_{t}^{2}\right)}{2} + \frac{u u_{tt} + u_{1} u1_{tt} + u1_{t}^{2} + u_{t}^{2}}{2}
\big)
+h^3\big(
\frac{\varepsilon u \left(u^{2} u_{ttt} + 9 u u_{tt} u_{t} + 6 u_{t}^{3}\right)}{6} + \frac{u u_{ttt} + u_{1} u1_{ttt} + 3 u1_{tt} u1_{t} + 3 u_{tt} u_{t}}{6}
\big)
+h^4\big(
\frac{\varepsilon \left(u^{3} u_{tttt} + 12 u^{2} u_{ttt} u_{t} + 9 u^{2} u_{tt}^{2} + 36 u u_{tt} u_{t}^{2} + 6 u_{t}^{4}\right)}{24} + \frac{u u_{tttt} + u_{1} u1_{tttt} + 4 u1_{ttt} u1_{t} + 3 u1_{tt}^{2} + 4 u_{ttt} u_{t} + 3 u_{tt}^{2}}{24}
\big)
+h^5\big(
\frac{\varepsilon \left(u^{3} u_{ttttt} + 15 u^{2} u_{tttt} u_{t} + 30 u^{2} u_{ttt} u_{tt} + 60 u u_{ttt} u_{t}^{2} + 90 u u_{tt}^{2} u_{t} + 60 u_{tt} u_{t}^{3}\right)}{120} + \frac{u u_{ttttt} + u_{1} u1_{ttttt} + 5 u1_{tttt} u1_{t} + 10 u1_{ttt} u1_{tt} + 5 u_{tttt} u_{t} + 10 u_{ttt} u_{tt}}{120}
\big)


In [47]:
f3 = NF(F3, [u.diff(t), u1.diff(t)], [f1, f2], head=False)
prn(f3, eps)
prnlatex(f3, eps)

h^0 =>


0

h^1 =>


0

h^2 =>


...

h^4 =>


0

h^5 =>


737*\varepsilon**3*u**7*u1/360000 + \varepsilon**2*u**3*u1*(737*u**2 - 2405*u1**2)/180000 + \varepsilon*u*u1*(737*u**2 - 1846*u1**2)/360000

0
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{737 \varepsilon^{3} u^{7} u_{1}}{360000} + \frac{\varepsilon^{2} u^{3} u_{1} \cdot \left(737 u^{2} - 2405 u_{1}^{2}\right)}{180000} + \frac{\varepsilon u u_{1} \cdot \left(737 u^{2} - 1846 u_{1}^{2}\right)}{360000}
\big)
